In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *
from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '137'
customer_name='%kimballmidwest%'
buckets = "Fixings & Fasteners"
formatted_attribute = 'yield_psi'

In [3]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['image_url']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

155


In [26]:
pat_psi=r'(?i)(psi)|(\d+\,\d+)|()'
df['psi']=df['Yield_P_S_I__'].apply(lambda x: re_extract(pat_psi, str(x)))
print(len(df[df['psi'].astype(str)=='[]']))
dfna=df[df['psi'].astype(str)=='[]']
dfna[curation_col]='n/a'
matchna=dfna[['external_id',curation_col]]

65


In [28]:
print(len(df[df['psi'].astype(str)!='[]']))
psi=df[df['psi'].astype(str)!='[]']
psi['psi']=psi['psi'].apply(lambda x: re.sub(r'\[','["',str(x))).apply(lambda x: re.sub(r'\]',' PSI"]',str(x))).apply(lambda x: re.sub(r"'",'',str(x)))
psi[curation_col]=psi['psi']
match_psi=psi[['external_id',curation_col]]
psi['psi'].explode().value_counts()
# match_psi

90


["56,000 PSI"]    10
["48,000 PSI"]    10
["80,000 PSI"]     8
["58,000 PSI"]     7
["72,000 PSI"]     7
["20,000 PSI"]     7
["60,000 PSI"]     7
["75,000 PSI"]     6
["90,000 PSI"]     5
["69,000 PSI"]     5
["54,000 PSI"]     4
["49,000 PSI"]     4
["78,100 PSI"]     4
["59,000 PSI"]     2
["50,000 PSI"]     2
["64,000 PSI"]     2
Name: psi, dtype: int64

In [ ]:
stop

# send to the folder for upload

In [32]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('Kimball Midwest', attribute, df, match_psi)

In [33]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('Kimball Midwest', attribute, df, matchna)